# Data Cleaning

Real world data is usually messy and can contain errors. These can be in the form of unnecessary (sometimes empty) columns, values not formatted correctly or recorded erroneously.

In this notebook we present a few data related issues and techniques on how to tackle these issues. We will first use the `student_debt.csv` data as previously.

In [ ]:
import pandas as pd

df = pd.read_csv("data/student_debt.csv")
df

Notice the column `Unnamed: 0` does not contain any useful information, it is a mismatched version of the index only. This happens often when the data is downloaded, created or otherwise transformed as a side effect. 

We can easily get rid of a column in Pandas with the `drop()` method. The drop method accepts a list of column names in the columns attribute. 
Furthermore, you can pass `inplace = True` to perform the operation on the dataframe. If you do not do this, the operation returns a new dataframe without the specified columns.

In [ ]:
df_dropped_column = df.drop(columns=["Unnamed: 0"]) #returns a new dataframe 
df.drop(columns=["Unnamed: 0"], inplace=True) #drops the column in the original dataframe
## Note that now df_dropped_column and df is exactly the same

Now that we have only the useful columns, we can look at the values in our data set. 
For this a convenient starting point is looking at whether all data types are as expected. 

In [ ]:
df.dtypes

Notice that `Period` has a data type object, whereas it should be an integer or float. 
If you look at some of the year values, they have an asterisk * next to them. This is because those figures are preliminary only, but we still need those years in raw number format, such that we can perform analysis on it.

We can use the `.apply()` function. This applies a custom function on every row of the dataframe its called upon.
For simpler functions we can write a lambda function. Don't let the fancy name fool you, they are actually pretty easy.

In python we can call the `replace()` method on any string to replace a piece of string to something user specified. We can use this to get rid of the * after the years. Then we can convert the years to integers with `int()`.

In [ ]:
# we specifiy a lambda function which applies a certain operation on every row of our dataframe
df.apply(lambda row: int(row['Period'].replace('*', '')), axis=1) 
#axis = 1 needed to apply the operations on every row, axis = 0 would apply the operations on every column, resulting in an error


Notice that it returns a new Pandas series, and does not modify the original dataframe. We can create or replace a column with pandas by simply specifying the column name and the values of the new column.

In [ ]:
df["Period"] = df.apply(lambda row:int(row['Period'].replace('*', '')), axis=1) 
df.dtypes

Notice that the data type of Period now changed to int64, which was our desired goal.

# Missing values and outliers on Salary data set

In some data sets you have missing or otherwise erroneous values. In general, missing values are very common in real life, and dealing with them is an important step in data cleaning.

For this we will use a different data set `DenBosch-avg-salary.csv`.

In [ ]:
df_salaries = pd.read_csv("data/DenBosch-avg-salary.csv")
df_salaries.iloc[40:50]

In [ ]:
df_salaries.dtypes

As you can see, part of this data has missing values (in the form of `.`) in several places. Furthermore also notice that all the numerical values are listed as objects, because Python uses a `.` to indicate decimals, and in this data set we have `,` separating them. Hence, Pandas cannot convert e.g. the number 40,2 to a number, only if we modify it to 40.2

First we have to do two things: replace the `,` to `.` in order to convert the numbers to `float`, and set the missing values to be actually missing, this latter can be done by filtering cells containing a `.` and setting their value to `np.nan`.

For this exercise, we select only the columns we want to clean. In our case, we only want columns with numbers, so we can drop *AreaCode, Neighbourhood code* columns for imputation.

Later on, we need to recombine (concatenate) the imputed columns with the other two we decided to drop.

In [ ]:
import numpy as np 

columns_not_impute = ['AreaCode', 'Neighbourhood code'] # columns we don't want to preprocess further, as they are not numerical
df_salaries_to_impute = df_salaries.drop(columns=columns_not_impute)

#iterate over all columns and replace , with . using a lambda function
for column in df_salaries_to_impute.columns:
    df_salaries_to_impute[column] = df_salaries_to_impute.apply(lambda row:row[column].replace(',', '.'), axis=1)

df_salaries_to_impute[df_salaries_to_impute.values == '.'] = np.nan #set the missing values to NaN

df_salaries_to_impute = df_salaries_to_impute.astype(float)


Now with the missing values we can do something called *missing value imputation*, which replaces missing values with some plausible value.
For this we will use the Python library scikit-learn, that has multiple methods to impute missing values.
In this tutorial we use the `SimpleImputer`, which imputes the average value of each column with missingness.
NOTE: This imputation algorithm only accepts a dataframe that contains one specific data type. If we specify `SimpleImputer(strategy = 'mean')` we can impute numerical values only. However, if we specify `SimpleImputer(strategy = 'median')` we can fill in the missing values for textual fields.


Before that we can look at how many missing values we have for each column, this can be done by using `isna()` that returns true if a cell is missing and False otherwise, and then `.sum()` over these values to get the number of missing values per column. In the code cell below we also divided by the number of rows, such that the values can be interpreted as percentages, but this is an optional step in this stage.

In [ ]:
# 
df_salaries_to_impute.isna().sum()/df_salaries_to_impute.shape[0]

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean') #initialize scikit learn imputer
imputed_values = imputer.fit_transform(df_salaries_to_impute)
### You can see that the imputation algorithm returns values only.
### We can convert this into a DataFrame by doing:
df_imputed_salaries = pd.DataFrame(imputed_values, columns=df_salaries_to_impute.columns, index=df_salaries_to_impute.index)
# Here we specify that we want a new dataframe wiht the imputed values and the same columns and index as the previous data frame

df_imputed_salaries

Now all we need to do is use `pd.concat()` to join the data with the discarded area codes.

In [ ]:
df_salaries_cleaned = pd.concat([df_salaries[columns_not_impute], df_imputed_salaries], axis=1)
df_salaries_cleaned

Finally, we can save the cleaned dataframe to a csv file, so that we can access it later. 
For this we can call the `to_csv()` method of a dataframe.
It is important that you specify `index = False` in the arguments, as we don't want to save indices, those are auto-generated by Pandas every time we open a new dataframe.

In [ ]:
df_salaries_cleaned.to_csv("data/cleaned-DenBosch-avg-salary.csv", index=False)

# Exercises

In these exercises you will perform similar cleaning tasks on several other data sets downloaded from our data dashboard.

The first data set we will use is the `municipality_data.csv`.

### Exercise 1

Load the data and check which parts need cleaning. Look for extra columns, mismatching data types, check the percentage of missing values. 


In [ ]:
#------------ Exercise code comes here ------------- #


### Exercise 2

Using the methods presented above, try and fix those inconsistencies. If you need to do missing value imputation, use the `SimpleImputer` with the correctly specified strategy.
Remember the steps how to fill in missing values:
- Check by hand which columns _should_ have numerical values.
- Check data types with `.dtypes` whether in fact they are numerical.
    - If not identify the cause (letters next to the numbers, dots for missing values instead of NaN etc.)
    - Replace/remove text from all columns that should be numerical
- Separate only numerical columns into a new dataframe
- Perform the imputation using the `SimpleImputer` 
- Merge (concatenate) the imputed numerical values with the (excluded) non-numerical values to get the original dataframe back

If you are stuck anywhere along the process, feel free to reach out to one of the Data mentors.

In [ ]:
# ------------ Exercise code comes here ------------- #

### Exercise 3

Save the cleaned dataframe to a new csv file.

In [ ]:
# ------------ Exercise code comes here ------------- #

## Cleaning second data set

Below we have some other practice material on another data set named `Farmers_NH3_emission.csv`.

### Exercise 4
Check the data types of the column. You should see that some data types do not match because of the units in the cells next to the numbers. Try to get rid of them and convert the columns to numbers (float or int as necessary).

If you don't know how to start, reach out to a Data mentor, we are happy to help!

As practice you can also check if the data contains any missing values.

In [ ]:
# ------------ Exercise code comes here ------------- #